In [ ]:
#Notebook to build adjacency graph of the genes with highest variability

In [1]:
import pandas as pd
import numpy as np
import scipy

In [22]:
#loading high_idx

import pickle

with open('high_var_genes', 'rb')as high_var_genes_file:
    high_var_genes = pickle.load(high_var_genes_file)

In [23]:
# for g in range(len(high_var_genes)):
#     high_var_genes[g] = high_var_genes[g].replace('.', '-')

In [4]:
pbmc = np.load('data/pbmc_normalized.npy')

In [ ]:
#For direct gene-gene correlation network 

pbmc_corr = abs(np.corrcoef(pbmc, rowvar=False))

In [32]:
np.save('data/adj_mat_direct_coex.npy', pbmc_corr)

In [15]:
links = pd.read_csv('data/9606.protein.links.full.v11.5.txt', delimiter=' ')

In [16]:
links.head()

,protein1,protein2,neighborhood,neighborhood_transferred,fusion,cooccurence,homology,coexpression,coexpression_transferred,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
0,9606.ENSP00000000233,9606.ENSP00000379496,0,0,0,0,0,0,54,0,0,0,0,103,85,155
1,9606.ENSP00000000233,9606.ENSP00000314067,0,0,0,0,0,0,0,0,180,0,0,0,61,197
2,9606.ENSP00000000233,9606.ENSP00000263116,0,0,0,0,0,0,62,0,152,0,0,0,101,222
3,9606.ENSP00000000233,9606.ENSP00000361263,0,0,0,0,0,0,0,0,161,0,0,47,58,181
4,9606.ENSP00000000233,9606.ENSP00000409666,0,0,0,0,0,60,63,0,213,0,0,0,72,270


In [17]:
names = pd.read_csv('data/9606.protein.info.v11.5.txt', delimiter='\t')

In [18]:
names.head()

,#string_protein_id,preferred_name,protein_size,annotation
0,9606.ENSP00000000233,ARF5,180,ADP-ribosylation factor 5; GTP-binding protein...
1,9606.ENSP00000000412,M6PR,277,Cation-dependent mannose-6-phosphate receptor;...
2,9606.ENSP00000001008,FKBP4,459,Peptidyl-prolyl cis-trans isomerase FKBP4; Imm...
3,9606.ENSP00000001146,CYP26B1,512,Cytochrome P450 26B1; Involved in the metaboli...
4,9606.ENSP00000002125,NDUFAF7,441,"Protein arginine methyltransferase NDUFAF7, mi..."


In [24]:
#converting gene symbols to ensembl IDs
ensembl_id = []
empty = []

for feature in high_var_genes:
    id = names.loc[names['preferred_name'] == feature, '#string_protein_id'].values
    if len(id) > 0:
        ensembl_id.append(id[0])
    else:
        empty.append(feature)


In [25]:
print(len(empty), len(ensembl_id))

201 1799


In [ ]:
for gene in empty:
    print(gene, high_var_genes.index(gene))

In [26]:
empty_idx = []

for gene in empty:
    empty_idx.append(high_var_genes.index(gene))

In [28]:
pbmc = np.delete(pbmc, empty_idx, axis=1)

In [29]:
pbmc.shape

(20000, 1799)

In [30]:
np.save('data/pbmc_normalized_filtered2.npy', pbmc)

In [31]:
ensembl_id[0:10]

['9606.ENSP00000413436',
 '9606.ENSP00000366525',
 '9606.ENSP00000369757',
 '9606.ENSP00000273550',
 '9606.ENSP00000370010',
 '9606.ENSP00000278572',
 '9606.ENSP00000341885',
 '9606.ENSP00000307889',
 '9606.ENSP00000362744',
 '9606.ENSP00000452780']

In [32]:
ensembl_dict = {}
i = 0

for id in ensembl_id:
    ensembl_dict[id] = i
    i += 1

In [33]:
filtered_links = links[(links['protein1'].isin(ensembl_id)) & (links['protein2'].isin(ensembl_id))]

In [34]:
filtered_links

,protein1,protein2,neighborhood,neighborhood_transferred,fusion,cooccurence,homology,coexpression,coexpression_transferred,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
9,9606.ENSP00000000233,9606.ENSP00000308413,0,0,0,0,0,0,0,0,150,0,0,0,55,162
13,9606.ENSP00000000233,9606.ENSP00000444357,0,0,0,0,0,85,64,270,0,0,0,0,0,320
17,9606.ENSP00000000233,9606.ENSP00000346733,0,0,0,0,0,0,0,0,147,0,0,0,86,187
22,9606.ENSP00000000233,9606.ENSP00000312150,0,0,0,0,0,0,65,0,127,0,0,0,50,156
28,9606.ENSP00000000233,9606.ENSP00000300026,0,0,0,0,0,59,62,0,169,0,0,0,69,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11938240,9606.ENSP00000485663,9606.ENSP00000262584,0,0,0,0,0,239,146,0,0,0,0,134,55,396
11938250,9606.ENSP00000485663,9606.ENSP00000383949,0,0,0,0,0,241,86,0,142,0,0,0,48,357
11938251,9606.ENSP00000485663,9606.ENSP00000346088,0,0,0,0,0,676,145,0,0,0,0,125,48,738
11938255,9606.ENSP00000485663,9606.ENSP00000335153,0,0,0,0,0,113,64,0,0,0,0,74,101,216


In [35]:
#convert edge list to adjacency matrix

adj = [[0]*len(ensembl_id) for x in range(len(ensembl_id))]

for edge in filtered_links.iterrows():
    idx_1 = ensembl_dict[edge[1]['protein1']]
    idx_2 = ensembl_dict[edge[1]['protein2']]
    adj[idx_1][idx_2] = edge[1]['coexpression']
    adj[idx_2][idx_1] = edge[1]['coexpression']

In [16]:
filtered_links.loc[(filtered_links['protein1'] == '9606.ENSP00000000233') & (filtered_links['protein2'] == '9606.ENSP00000308413')]

,protein1,protein2,neighborhood,neighborhood_transferred,fusion,cooccurence,homology,coexpression,coexpression_transferred,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
9,9606.ENSP00000000233,9606.ENSP00000308413,0,0,0,0,0,0,0,0,150,0,0,0,55,162


In [17]:
filtered_links.loc[(filtered_links['protein2'] == '9606.ENSP00000308413') & (filtered_links['protein1'] == '9606.ENSP00000000233')]

,protein1,protein2,neighborhood,neighborhood_transferred,fusion,cooccurence,homology,coexpression,coexpression_transferred,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
9,9606.ENSP00000000233,9606.ENSP00000308413,0,0,0,0,0,0,0,0,150,0,0,0,55,162


In [36]:
adj_mat = np.asarray(adj, dtype='int32')

In [22]:
np.save('data/adj_mat_on_normalized2.npy', adj_mat)